In [94]:
date_number = set()
for i in product[['공급일자', '회원번호']].values:
    date_number.add(tuple(i))

In [95]:
cart = pd.DataFrame(columns=['공급일자', '회원번호', '장바구니', '구매수량합계', '구매금액합계', '구매매장'])

for num, data in tqdm(enumerate(date_number)):
    cart.loc[num, '공급일자'] = data[0]
    cart.loc[num, '회원번호'] = data[1]
    temp = product.loc[(product['공급일자'] == data[0]) & (product['회원번호'] == data[1])]
    cart.loc[num, '장바구니_대'] = '+'.join(temp['물품대분류'])
    cart.loc[num, '장바구니_중'] = '+'.join(temp['물품중분류'])
    cart.loc[num, '장바구니_소'] = '+'.join(temp['물품소분류'])
    cart.loc[num, '구매수량합계'] = temp['구매수량'].sum()
    cart.loc[num, '구매금액합계'] = temp['구매금액'].sum()
    cart.loc[num, '구매매장'] = '+'.join(temp['구매매장'])

In [96]:
cart.to_csv('장바구니분석.csv', index=False)

# 윗부분은 실행시키지 말것

In [8]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [9]:
product = pd.read_csv('product_preprocessed.csv')
member = pd.read_csv('member_data_preprocessed.csv')

In [10]:
cart = pd.read_csv('장바구니분석.csv')

In [11]:
cart = pd.merge(member[['회원번호', '연령대', '동']], cart, on='회원번호')

In [12]:
cart.head()

,회원번호,연령대,동,공급일자,장바구니,구매수량합계,구매금액합계,구매매장,장바구니_대,장바구니_중,장바구니_소
0,1000012800,55 ~ 64,풍덕천동,165,NaN,2.0,7658,매장B,반찬,두부/유부,두부
1,1000103744,34 이하,고림동,286,NaN,7.0,36948,매장C+매장C+매장C+매장C+매장C+매장C,간식+간식+간식+간식+간식+간식,식사대용+식사대용+과자+과자+식사대용+과자,식사대용/기타+식사대용/기타+스낵+스낵+식사대용/기타+스낵
2,1000122304,35 ~ 44,기타,278,NaN,2.0,13879,매장C+매장C,축산물+반찬,알+두부/유부,유정란+두부
3,1000122304,35 ~ 44,기타,276,NaN,1.0,1914,매장C,반찬,두부/유부,두부
4,1000122304,35 ~ 44,기타,104,NaN,3.0,33885,매장B+매장B+매장B,양념/가루+축산물+양념/가루,가루+알+가루,프리믹스+유정란+프리믹스


In [13]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['연령대'] == '34 이하', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=v_min_sup, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
1,0.127,(두부/유부)
0,0.124,(과자)
9,0.112,(음료)
10,0.088,(잎/줄기채소)
5,0.082,(알)
8,0.071,(유제품)
7,0.063,(열매채소)
2,0.059,(밀가공)
3,0.053,(버섯)
4,0.053,(빵)


In [14]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['연령대'] == '35 ~ 44', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=v_min_sup, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
1,0.126,(두부/유부)
4,0.106,(알)
0,0.101,(과자)
9,0.101,(음료)
10,0.086,(잎/줄기채소)
7,0.073,(유제품)
2,0.065,(밀가공)
3,0.064,(빵)
5,0.061,(양념채소)
8,0.060,(육가공)


In [15]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['연령대'] == '45 ~ 54', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=v_min_sup, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
2,0.135,(두부/유부)
5,0.114,(알)
11,0.112,(잎/줄기채소)
7,0.076,(열매채소)
6,0.073,(양념채소)
1,0.069,(과자)
3,0.066,(밀가공)
8,0.063,(유제품)
10,0.062,(음료)
4,0.060,(빵)


In [16]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['연령대'] == '55 ~ 64', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=v_min_sup, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
2,0.150,(두부/유부)
11,0.138,(잎/줄기채소)
6,0.112,(알)
8,0.088,(열매채소)
7,0.082,(양념채소)
1,0.068,(과자)
12,0.062,(콩나물)
5,0.057,(뿌리채소)
9,0.057,(유제품)
3,0.057,(밀가공)


In [17]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = cart.loc[cart['연령대'] == '65 이상', '장바구니_중'].str.split('+'))

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)

v_min_sup = 0.05
df_freq = apriori(df_asso, min_support=v_min_sup, use_colnames=True).sort_values('support', ascending=False)
df_freq.round(3)

,support,itemsets
2,0.150,(두부/유부)
8,0.143,(잎/줄기채소)
4,0.120,(알)
6,0.094,(열매채소)
5,0.084,(양념채소)
3,0.062,(뿌리채소)
1,0.058,(과자)
0,0.057,(과일)
7,0.051,(유제품)
9,0.051,(콩나물)


In [130]:
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.05)
df_asso_rule.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(축산물),(채소),0.264052,0.437043,0.086790,0.328686,0.752067,-0.028612,0.838589
5,(간식),(채소),0.192627,0.437043,0.060576,0.314475,0.719552,-0.023610,0.821206
3,(반찬),(채소),0.210054,0.437043,0.065387,0.311288,0.712259,-0.026415,0.817405
0,(채소),(축산물),0.437043,0.264052,0.086790,0.198585,0.752067,-0.028612,0.918311
2,(채소),(반찬),0.437043,0.210054,0.065387,0.149612,0.712259,-0.026415,0.928925
4,(채소),(간식),0.437043,0.192627,0.060576,0.138605,0.719552,-0.023610,0.937286
